In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

# Q1
Apply histogram equalization on a histogram equalized image. In other words, apply histogram equalization twice on an image. Is there any difference between the first and second output image? You are free to use any image.

In [2]:
img = cv.imread('winter.jfif', 0)

eq = cv.equalizeHist(img)

eq1 = cv.equalizeHist(eq)

cv.imshow('image', np.hstack((eq, eq1)))
cv.waitKey(0)
cv.destroyAllWindows()

There seems to be no difference in both images 

# Q2
Load the image 'electronic.jfif'. Then,
* Experiment with different kernel size. Which kernel size is the most appropriate?
* Perform edge detection using Sobel operator (combined both x and y) with and without image smoothing (Gaussian blurring). Display the 2 images.
* Try Laplacian of Gaussian

Based on the exercise conducted from (A) - (C), which image processing pathway is optimal? Justify your answer.

In [3]:
img = cv.imread('electronic.jfif', 0)

# with smoothing
blur = cv.GaussianBlur(img, (5, 5), 0)
sobelx = cv.Sobel(blur, cv.CV_64F, 1, 0, ksize = 3)
sobely = cv.Sobel(blur, cv.CV_64F, 0, 1, ksize = 3)

grad_mag_L2 = cv.magnitude(sobelx, sobely)
grad_mag_L2 = cv.convertScaleAbs(grad_mag_L2)

# without smoothing
sobelx2 = cv.Sobel(img, cv.CV_64F, 1, 0, ksize = 3)
sobely2 = cv.Sobel(img, cv.CV_64F, 0, 1, ksize = 3)

grad_mag_L22 = cv.magnitude(sobelx2, sobely2)
grad_mag_L22 = cv.convertScaleAbs(grad_mag_L22)

# laplacian
laplacian = cv.Laplacian(blur, cv.CV_64F, ksize = 3)

laplacian_8u = cv.convertScaleAbs(laplacian)

cv.imshow('img', np.hstack((grad_mag_L2, grad_mag_L22, laplacian_8u)))

cv.waitKey(0)
cv.destroyAllWindows()

The kernel_size of 3 is the most appropriate. Kernel_size of 1 could work as well but the color of the edge lines are too light.    
The method of performing smoothing and then using sobel is the most optimal pathway as the edges detected is the most precise and clear. Smoothing blurs the imgae to remove noise and small edges that is unimportant so that it will not be detected during edge detection.

# Q3
Experiment with different edge detectors: Sobel, Laplacian, Prewitt, Scharr derivatives and Canny operators (all with aperture size of 3) on image named 'pineapple.jfif'. Comment on the results.

In [4]:
img = cv.imread('pineapple.jfif', 0)

# Sobel
blur = cv.GaussianBlur(img, (5, 5), 0)
sobelx = cv.Sobel(blur, cv.CV_64F, 1, 0, ksize = 3)
sobely = cv.Sobel(blur, cv.CV_64F, 0, 1, ksize = 3)

grad_mag_L2 = cv.magnitude(sobelx, sobely)
grad_mag_L2 = cv.convertScaleAbs(grad_mag_L2)

# Laplacian 
laplacian = cv.Laplacian(blur, cv.CV_64F, ksize = 3)
laplacian_8u = cv.convertScaleAbs(laplacian)

# Prewitt
kernelx = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
kernely = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
prewittx = cv.filter2D(blur, -1, kernelx)
prewitty = cv.filter2D(blur, -1, kernely)

# Scharr
scharrx = cv.Scharr(blur, cv.CV_64F, 1, 0)
scharry = cv.Scharr(blur, cv.CV_64F, 0, 1)

grad_mag_L2_scharr = cv.magnitude(scharrx, scharry)//4
grad_mag_L2_scharr = cv.convertScaleAbs(grad_mag_L2_scharr)

# Canny
output_canny = cv.Canny(blur, 50, 100, 3)

cv.imshow('results', np.hstack((grad_mag_L2, laplacian_8u, prewittx + prewitty, grad_mag_L2_scharr, output_canny)))
cv.waitKey(0)
cv.destroyAllWindows()

* The image showed by sobel method can be seen as a standard baseline for the edge detection.      
* The image showed by laplacian method seems to be underwhelming since it detects much lesser edges when compared to sobel.     
* The image showed by prewitt method has quite similar results to laplacian but the lines of the edges are blurry.
* The image showed by scharr method can detect the edges in the image pretty well, it is devided by 4 to reduce the thickness of the lines drawn at the edges. The have achieved similar results with sobel method.     
* The image showed by canny method is the most clear and easy to recognize, the edges are outlines very well.

# Q4
Write a program to identify the white object (probably laptop) present in the image 'electronic.jfif'. Draw bounding boxes on the objects.

In [5]:
img = cv.imread('electronic.jfif', 0)
bfilter = cv.bilateralFilter(img, 10, 75, 75)

_, th = cv.threshold(bfilter, 200, 255, cv.THRESH_BINARY)

contour, hierarchy = cv.findContours(th, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

img_bgr = cv.cvtColor(img, cv.COLOR_GRAY2BGR)

for i in range(len(contour)):
    cnt = contour[i]
    x, y, w, h = cv.boundingRect(cnt)
    cv.rectangle(img_bgr, (x,y), (x+w, y+h), (0, 255, 0), 2)

cv.imshow('contour', img_bgr)
cv.waitKey(0)
cv.destroyAllWindows()